## Day 47 Lecture 1 Assignment

In this assignment, we will apply k-means clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [4]:
# answer goes here
data = 'https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/nba_player_seasons.csv'
df = pd.read_csv(data)


In [8]:
df

,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2019,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,0.357,1.3,4.1,0.323,0.5,1.0,0.500,0.487,0.4,0.4,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,2019,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,0.222,0.2,1.5,0.133,0.2,0.3,0.667,0.278,0.7,1.0,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,2019,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,0.345,0.7,2.2,0.338,0.4,1.1,0.361,0.459,0.2,0.3,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,0.595,0.0,0.0,0.000,6.0,10.1,0.596,0.595,1.8,3.7,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,0.576,0.0,0.2,0.200,3.4,5.7,0.588,0.579,2.0,2.8,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2136,2016,Joe Young,PG,23,IND,41,0,9.4,1.5,4.1,0.367,0.2,1.1,0.217,1.3,3.0,0.423,0.396,0.5,0.6,0.800,0.1,1.1,1.2,1.6,0.4,0.0,0.8,0.7,3.8
2137,2016,Nick Young,SG,30,LAL,54,2,19.1,2.3,6.9,0.339,1.4,4.4,0.325,0.9,2.5,0.363,0.442,1.2,1.4,0.829,0.3,1.5,1.8,0.6,0.4,0.1,0.6,0.9,7.3
2138,2016,Thaddeus Young,PF,27,BRK,73,73,33.0,6.8,13.2,0.514,0.1,0.4,0.233,6.7,12.8,0.523,0.518,1.4,2.2,0.644,2.4,6.6,9.0,1.8,1.5,0.5,1.9,2.5,15.1
2139,2016,Cody Zeller,C,23,CHO,73,60,24.3,3.2,6.0,0.529,0.0,0.1,0.100,3.2,5.8,0.539,0.530,2.4,3.2,0.754,1.9,4.3,6.2,1.0,0.8,0.9,0.9,2.8,8.7


The goal is to cluster these player-seasons to identify potential player "archetypes".

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

In [14]:
# answer goes here

df20st = df[df.GS > 19]
df10min = df20st[df20st.MP > 9.9]
df10min

,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,0.595,0.0,0.0,0.000,6.0,10.1,0.596,0.595,1.8,3.7,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,0.576,0.0,0.2,0.200,3.4,5.7,0.588,0.579,2.0,2.8,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
7,2019,LaMarcus Aldridge,C,33,SAS,81,81,33.2,8.4,16.3,0.519,0.1,0.5,0.238,8.3,15.8,0.528,0.522,4.3,5.1,0.847,3.1,6.1,9.2,2.4,0.5,1.3,1.8,2.2,21.3
10,2019,Jarrett Allen,C,20,BRK,80,80,26.2,4.2,7.1,0.590,0.1,0.6,0.133,4.1,6.5,0.629,0.595,2.5,3.5,0.709,2.4,6.0,8.4,1.4,0.5,1.5,1.3,2.3,10.9
12,2019,Al-Farouq Aminu,PF,28,POR,81,81,28.3,3.2,7.3,0.433,1.2,3.5,0.343,2.0,3.9,0.514,0.514,1.9,2.1,0.867,1.4,6.1,7.5,1.3,0.8,0.4,0.9,1.8,9.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2122,2016,Deron Williams,PG,31,DAL,65,63,32.4,4.9,11.9,0.414,1.5,4.3,0.344,3.4,7.6,0.454,0.477,2.8,3.2,0.869,0.3,2.6,2.9,5.8,0.9,0.2,2.3,2.4,14.1
2125,2016,Lou Williams,SG,29,LAL,67,35,28.5,4.2,10.3,0.408,1.6,4.8,0.344,2.6,5.6,0.464,0.488,5.3,6.3,0.830,0.5,2.1,2.5,2.5,0.9,0.3,1.6,1.6,15.3
2126,2016,Marvin Williams,PF,29,CHO,81,81,28.9,4.2,9.2,0.452,1.9,4.7,0.402,2.3,4.6,0.504,0.554,1.5,1.8,0.833,1.6,4.9,6.4,1.4,0.7,1.0,0.8,1.6,11.7
2138,2016,Thaddeus Young,PF,27,BRK,73,73,33.0,6.8,13.2,0.514,0.1,0.4,0.233,6.7,12.8,0.523,0.518,1.4,2.2,0.644,2.4,6.6,9.0,1.8,1.5,0.5,1.9,2.5,15.1


In [15]:
df10min.columns

Index(['Seas', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [51]:
# answer goes here
X = df10min[['PTS','TRB','AST','STL','BLK','TOV','FG%','G']]




Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [52]:
# answer goes here
scale = StandardScaler()
scaled = pd.DataFrame(scale.fit_transform(X), columns=X.columns)

In [53]:
scaled

,PTS,TRB,AST,STL,BLK,TOV,FG%,G
0,0.114638,1.612909,-0.617816,1.393220,0.824001,0.023511,2.168189,0.888685
1,-0.744985,0.768627,-0.330886,-0.050271,0.420162,-0.218831,1.849645,1.044434
2,1.386880,1.497780,-0.235242,-1.012598,1.429758,0.144681,0.894013,0.966560
3,-0.401136,1.190768,-0.713459,-1.012598,1.833596,-0.461172,2.084361,0.888685
4,-0.659023,0.845380,-0.761281,-0.290853,-0.387514,-0.945854,-0.547818,0.966560
...,...,...,...,...,...,...,...,...
799,0.149023,-0.919937,1.390695,-0.050271,-0.791352,0.750535,-0.866362,-0.279439
800,0.355332,-1.073443,-0.187421,-0.050271,-0.589433,-0.097660,-0.966955,-0.123689
801,-0.263596,0.423239,-0.713459,-0.531435,0.824001,-1.067025,-0.229274,0.966560
802,0.320947,1.421027,-0.522172,1.393220,-0.185595,0.265852,0.810185,0.343560


Run K-means clustering with K = 3 and print out the resulting centroids. When printing the centroids, transform the scaled centroids back into their corresponding unscaled values. What "archetypes" do you see?

In [60]:
# answer goes here

kmeans = KMeans(n_clusters=3)
kmeans.fit_predict(scaled)
clust = scale.inverse_transform(kmeans.cluster_centers_)
clust = pd.DataFrame(clust, columns=scaled.columns)
clust.style.background_gradient()



,PTS,TRB,AST,STL,BLK,TOV,FG%,G
0,12.656995,8.136269,1.750777,0.713990,1.181865,1.556995,0.539865,68.378238
1,10.841026,3.981818,2.140793,0.828671,0.360373,1.266434,0.438406,68.617716
2,19.482967,5.386813,5.872527,1.357692,0.512088,2.787912,0.451280,68.741758


Experiment with different values of K. Do any further interesting archetypes come out?

In [61]:
# answer goes here
kmeans1 = KMeans(n_clusters=2)
kmeans1.fit_predict(scaled)
clust1 = scale.inverse_transform(kmeans1.cluster_centers_)
clust1 = pd.DataFrame(clust1, columns=scaled.columns)
clust1.style.background_gradient()



,PTS,TRB,AST,STL,BLK,TOV,FG%,G
0,19.475829,5.991943,5.517536,1.319431,0.639336,2.736967,0.457488,68.606635
1,11.011973,5.049916,1.957673,0.779089,0.575042,1.304722,0.468589,68.581788


In [62]:
kmeans = KMeans(n_clusters=6)
kmeans.fit_predict(scaled)
clust = scale.inverse_transform(kmeans.cluster_centers_)
clust = pd.DataFrame(clust, columns=scaled.columns)
clust.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV,FG%,G
0,20.061240,5.248837,6.366667,1.422481,0.450388,2.905426,0.451512,71.837209
1,8.238636,3.250568,1.610795,0.657955,0.326705,0.950568,0.432494,70.926136
2,9.937500,6.727941,1.444853,0.600735,0.962500,1.236765,0.539978,69.580882
3,13.745854,4.799512,2.718049,1.029756,0.385366,1.598537,0.446322,74.029268
4,12.843333,4.327778,3.233333,0.911111,0.415556,1.738889,0.431211,43.500000
5,18.769118,10.607353,2.582353,0.975000,1.661765,2.304412,0.533779,71.191176


In [63]:
kmeans = KMeans(n_clusters=8)
kmeans.fit_predict(scaled)
clust = scale.inverse_transform(kmeans.cluster_centers_)
clust = pd.DataFrame(clust, columns=scaled.columns)
clust.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV,FG%,G
0,7.340323,3.214516,1.308871,0.590323,0.329032,0.841129,0.432097,65.137097
1,24.602941,8.091176,7.714706,1.555882,0.800000,3.891176,0.479000,71.588235
2,17.372581,10.440323,2.132258,0.911290,1.738710,2.054839,0.544677,72.532258
3,11.389744,3.376923,2.756410,0.947436,0.261538,1.407051,0.430615,75.141026
4,13.894366,4.408451,3.671831,0.995775,0.429577,1.942254,0.434930,42.563380
5,18.800855,4.581197,5.591453,1.329060,0.383761,2.559829,0.445530,71.940171
6,13.902941,6.405147,2.156618,0.936765,0.647794,1.517647,0.464324,73.588235
7,9.249038,6.644231,1.250962,0.542308,0.921154,1.191346,0.552269,67.000000
